# Upgrading from MathProgBase.jl to MathOptInterface.jl

+ MathProgBase.jl has been [depreciated](https://github.com/JuliaOpt/MathProgBase.jl/pull/231) since Jan 2019. It is therefore not officially supported by julia v1.2 and above. We should use [MathOptInterface.jl](https://github.com/jump-dev/MathOptInterface.jl) instead. 
+ Other packages such as BinaryProvider.jl and Ipopt.jl needs to also be updated to the latest version or depreciated and use the latest one supported by the Julia community.
+ To ensure a smooth update, checkout the [official documentation](https://jump.dev/MathOptInterface.jl/dev/tutorials/mathprogbase/) as well as examples by [WiSER.jl](https://github.com/OpenMendel/WiSER.jl/blob/master/src/fit.jl) and [OrdinalMultinomialModels.jl](https://github.com/OpenMendel/OrdinalMultinomialModels.jl/blob/master/src/ordmnfit.jl)

In [26]:
using Revise
using Ipopt
using QuasiCopula
using Random
using GLM
using LinearAlgebra
using Statistics
using StatsBase
using MathOptInterface

ENV["COLUMNS"] = 240
BLAS.set_num_threads(1)
Threads.nthreads()

k = 0 # number of causal SNPs

qc_model, G, βtrue, θtrue, γtrue, τtrue = simulate_longitudinal_traits(
    n = 10000, # sample size
    d_min = 5, # min number of observations per sample
    d_max = 5, # max number of observations per sample
    p = 15, # number of fixed effects, including intercept
    m = 2, # number of variance components
    q = 1000, # number of SNPs
    k = k, # number of causal SNPs
    seed = 2024,
    y_distribution = Bernoulli,
    τtrue = 0.5,
    T = Float64,
    maf = 0.3,
    causal_snp_β = 0.2
)

@show qc_model;

@time optm = QuasiCopula.fit!(qc_model);

qc_model = Quasi-Copula Variance Component Model
  * base distribution: Bernoulli
  * link function: LogitLink
  * number of clusters: 10000
  * cluster size min, max: 5, 5
  * number of variance components: 2
  * number of fixed effects: 15



LoadError: UndefVarError: `NLPBlock` not defined

In [24]:
@show βtrue
@show qc_model.β
@show qc_model.∇β

@show θtrue
@show qc_model.θ
@show qc_model.∇θ;

# @show τtrue
# @show qc_model.τ
# @show qc_model.∇τ;

βtrue = [1.0, -0.5, 0.5, 0.5, 0.5, -0.5, 0.5, -0.5, 0.5, -0.5, -0.5, -0.5, 0.5, -0.5, -0.5]
qc_model.β = [1.0117570318510478, -0.5089169834363244, 0.49576633351557, 0.48814105668564467, 0.5059336670702058, -0.497950756571916, 0.4880296147654393, -0.4960080041746038, 0.4962344136852955, -0.4898931816925872, -0.509985681681852, -0.4832691011406199, 0.4787231797420944, -0.51456972551484, -0.47154951845097376]
qc_model.∇β = [-0.0007123036282812856, -6.826599935205735e-5, 0.0001239131191604903, 0.0005152871972469164, 0.00028022120171788956, 3.6481156232726075e-5, 7.566351851695519e-5, 0.0002129179100764933, 0.00044149755171069005, 0.0001915248276890269, -0.0002159099408911569, 0.00034448697090094793, 9.044316783501927e-6, -8.450373955692925e-6, -0.0004037025329033106]
θtrue = [0.1, 0.1]
qc_model.θ = [0.1107015363240431, -9.998245502613854e-9]
qc_model.∇θ = [-0.001189266278714296, -15.137726286741007]


In [2]:
import QuasiCopula.config_solver
import QuasiCopula.initialize_model!
import QuasiCopula.modelpar_to_optimpar!
T = Float64
const MOI = MathOptInterface
solver = Ipopt.Optimizer()
solver_config = Dict("print_level"           => 5, 
                 "mehrotra_algorithm"    => "yes",
                 "warm_start_init_point" => "yes",
                 "max_iter"              => 100)
solvertype = typeof(solver)




    solvertype = typeof(solver)
    solvertype <: Ipopt.Optimizer ||
        @warn("Optimizer object is $solvertype, `solver_config` may need to be defined.")
    config_solver(solver, solver_config)

    # set lower bounds and upper bounds of parameters
    npar = gcm.p + gcm.m
    lb   = fill(-Inf, npar)
    ub   = fill(Inf, npar)
    offset = gcm.p + 1
    for k in 1:gcm.m
        lb[offset] = 0
        offset += 1
    end

    # set up NLP optimization problem
    MOI.empty!(solver)
    NLPBlock = MOI.NLPBlockData(
        MOI.NLPBoundsPair.(lb, ub), gcm, true
    )
    par0 = Vector{T}(undef, npar)
    solver_pars = MOI.add_variables(solver, npar)
    for i in 1:npar
        MOI.set(solver, MOI.VariablePrimalStart(), solver_pars[i], par0[i])
    end
    MOI.set(solver, MOI.NLPBlock(), NLPBlock)
    MOI.set(solver, MOI.ObjectiveSense(), MOI.MIN_SENSE)

    # initial condition
    QuasiCopula.initialize_model!(gcm)
    QuasiCopula.modelpar_to_optimpar!(par0, gcm)
    for i in 1:npar
        MOI.set(solver, MOI.VariablePrimalStart(), solver_pars[i], par0[i])
    end

    MOI.optimize!(solver)


LoadError: UndefVarError: `config_solver` not defined

## TrajGWAS.jl test

In [ ]:
using CSV, DataFrames, WiSER
filepath = normpath(joinpath(dirname(pathof(WiSER)), "../data/"))
df = DataFrame(CSV.File(filepath * "sbp.csv"))

m = WSVarLmmModel(
    @formula(sbp ~ 1 + agegroup + gender + bmi_std + meds), 
    @formula(sbp ~ 1 + bmi_std), 
    @formula(sbp ~ 1 + agegroup + meds + bmi_std),
    :id, df);

solver = Ipopt.Optimizer()


    solvertype = typeof(solver)
    solvertype <: Ipopt.Optimizer ||
        @warn("Optimizer object is $solvertype, `solver_config` may need to be defined.")
    # Pass options to solver
    WiSER.config_solver(solver, solver_config)
    # set up NLP optimization problem
    npar = m.l + ◺(m.q)
    MOI.empty!(solver)
    lb   = T[]
    ub   = T[]

    NLPBlock = MOI.NLPBlockData(
        MOI.NLPBoundsPair.(lb, ub), m, true
    )

    par0 = Vector{T}(undef, npar)
    WiSER.modelpar_to_optimpar!(par0, m)

    solver_pars = MOI.add_variables(solver, npar)
    for i in 1:npar
        MOI.set(solver, MOI.VariablePrimalStart(), solver_pars[i], par0[i])
    end

    MOI.set(solver, MOI.NLPBlock(), NLPBlock)
    MOI.set(solver, MOI.ObjectiveSense(), MOI.MIN_SENSE)
